In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser 
from langchain_community.document_loaders.csv_loader import CSVLoader
import os  # 환경 변수 설정을 위한 모듈

# OpenAI API 키 설정 (실제 사용 시에는 안전한 방법으로 API 키를 관리해야 함)
os.environ["OPENAI_API_KEY"] = ''

In [16]:
# 1. CSV 파일 로드
# - CSVLoader를 사용하여 지정된 경로의 CSV 파일을 로드
# - 각 행(row)이 하나의 문서(Document)로 변환됨
loader = CSVLoader(file_path="../data/titanic.csv")
docs = loader.load()
print(f"Row 수: {len(docs)}")

Row 수: 891


In [17]:
# 2. 불러온 문서 내용 확인
# - 특정 문서의 내용을 출력하여 데이터 확인
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")


[페이지내용]
PassengerId: 11
Survived: 1
Pclass: 3
Name: Sandstrom, Miss. Marguerite Rut
Sex: female
Age: 4
SibSp: 1
Parch: 1
Ticket: PP 9549
Fare: 16.7
Cabin: G6
Embarked: S

[metadata]
{'source': '../data/titanic.csv', 'row': 10}



In [18]:
# 3. 문서를 작은 청크로 분할
# - RecursiveCharacterTextSplitter를 사용하여 텍스트를 일정 크기의 청크로 나눔
# - chunk_size: 한 개의 청크 크기 (문자 수)
# - chunk_overlap: 청크 간 겹치는 부분의 크기 (연속성 유지)
chunk_size = 1000  # 적절한 값 조절 필요
chunk_overlap = 100  # 적절한 값 조절 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# 문서를 여러 개의 작은 청크로 나눔
splits = text_splitter.split_documents(docs)
print(f"청크 개수: {len(splits)}")  # 생성된 청크 개수 확인

청크 개수: 891


In [19]:
# 4. 벡터스토어(Vector Store) 생성
# - FAISS(Facebook AI Similarity Search)를 사용하여 벡터 DB를 생성
# - OpenAIEmbeddings를 사용하여 텍스트를 벡터로 변환하여 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [20]:
# 5. 문서에서 정보를 검색할 수 있도록 검색기(retriever) 생성
retriever = vectorstore.as_retriever()

In [21]:
# 6. 프롬프트 템플릿 정의 (출처 포함)
# - 검색된 문서를 기반으로 질문에 답변하도록 설계
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.
검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.
#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [26]:
# 7. LLM 모델 선택
# - OpenAI의 GPT-4o-mini 모델을 사용하여 검색된 정보를 바탕으로 응답을 생성
# - temperature=0 설정으로 일관된 답변을 생성하도록 설정
llm = ChatOpenAI(model_name="gpt-5-nano",temperature=0)

In [27]:
# 8. RAG(Retrieval-Augmented Generation) 체인 생성
# - 검색된 문서를 바탕으로 LLM이 답변을 생성하도록 연결
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # 검색을 수행하는 객체 입력
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "데이터에 대해서 알려주세요."
response = rag_chain.invoke(user_question)  # 사용자의 질문을 입력
print(response)  # 최종 생성된 답변 출력

다음 문맥에서 볼 수 있는 데이터는 Titanic 데이터셋의 승객 정보로 보입니다. 주요 열과 몇 가지 특징은 아래와 같습니다.

- 열(컬럼) 예시
  - PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
  - Survived는 생존 여부를 나타내는 이진 값(1=생존, 0=사망)
  - Pclass는 승객의 등급(1=1등석, 2=2등석, 3=3등석)
  - Age는 나이(없는 값도 있음)
  - SibSp는 함께 탑승한 형제/배우자 수, Parch는 부모/자식 동반 수
  - Fare는 요금
  - Embarked는 탑승 항구(S, C, Q 등)
  - Ticket, Cabin은 티켓 정보와 선실 정보

- 샘플 데이터 특징 (주어진 4개 문서의 예시)
  - PassengerId: 644 / Survived: 1 / Pclass: 3 / Name: Foo, Mr. Choong / Sex: male / Age: (비어 있음) / SibSp: 0 / Parch: 0 / Ticket: 1601 / Fare: 56.4958 / Cabin: (비어 있음) / Embarked: S
  - PassengerId: 170 / Survived: 0 / Pclass: 3 / Name: Ling, Mr. Lee / Sex: male / Age: 28 / SibSp: 0 / Parch: 0 / Ticket: 1601 / Fare: 56.4958 / Cabin: (비어 있음) / Embarked: S
  - PassengerId: 839 / Survived: 1 / Pclass: 3 / Name: Chip, Mr. Chang / Sex: male / Age: 32 / SibSp: 0 / Parch: 0 / Ticket: 1601 / Fare: 56.4958 / Cabin: (비어 있음) / Embarked: S
  - PassengerId: 75 / Survived:

In [29]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "이 데이터를 가지고 어떤 걸 분석할 수 있을지 아이디어 5개 리스트 형식으로 정리해줘."
response = rag_chain.invoke(user_question)  # 사용자의 질문을 입력
print(response)  # 최종 생성된 답변 출력

- 아이디어 1: Survived를 종속 변수로, Pclass, Sex, Embarked에 따른 생존율 분석
  - 각 변수 조합별 교차표와 생존율을 비교하고, 어떤 조합에서 생존 확률이 높거나 낮은지 파악하기.

- 아이디어 2: Fare와 Survived의 관계 분석
  - Fare 구간별 생존율을 확인하고, Fare가 높을수록 생존 확률에 어떤 패턴이 있는지 시각화/통계로 파악하기. 필요 시 로지스틱 회귀로 계수 해석하기.

- 아이디어 3: 가족 규모가 생존에 미치는 영향 분석
  - SibSp와 Parch를 합쳐 FamilySize를 만들고, FamilySize별 Survived 비율과 분포를 비교. 가족 규모가 생존에 주는 영향 여부를 탐색하기.

- 아이디어 4: Age와 생존의 관계 분석
  - Age가 존재하는 경우의 생존율을 연령대별로 비교하고, Age가 결측인 케이스에 대한 처리 방법(대체값, 제거 등)을 함께 제시하기.

- 아이디어 5: Embarked 포트별 특성 비교 및 예측 모델 아이디어
  - Embarked에 따른 Survived 비율과 Fare 평균 차이를 분석하고, 간단한 baseline 모델(예: 로지스틱 회귀)의 특징으로 Pclass, Sex, Age, Fare, Embarked 등을 사용해 Survived를 예측하는 아이디어를 구상하기.
